In [2]:
!pip install pendulum

In [54]:
import modules.s3_module as s3_module
import re
import pyarrow.fs as fs
from modules.config import Musinsa_Config
import pyarrow.parquet as pq

s3 = s3_module.connect_s3fs()

def connect_s3fs():
    s3 = fs.S3FileSystem(
        access_key=AWS_Config.AWS_ACCESS_KEY_ID,
        secret_key=AWS_Config.AWS_SECRET_ACCESS_KEY,
        region=AWS_Config.REGION,
        endpoint_override='https://s3.ap-northeast-2.amazonaws.com'  
    )
    return s3

def get_product_ids(s3_path):
    dataset = pq.ParquetDataset(s3_path, filesystem=connect_s3fs())
    table = dataset.read(columns=['product_id'])
    
    product_ids = table['product_id'].to_pylist()
    
    return product_ids

bucket_path = "project4-silver-data/"

file_key = f"{Musinsa_Config.today_date}/Musinsa/RankingData/전체/"

path = bucket_path + file_key

files = s3.get_file_info(fs.FileSelector(base_dir=path, recursive=True))

category4depth = "스니커즈"
# 패턴에 맞는 파일 경로 리스트 생성

directory_pattern = re.compile(rf'.*{category4depth}\.parquet$')
directories = [file for file in files if file.type == fs.FileType.Directory and directory_pattern.match(file.path)]

product_ids = []
for directory in directories:
    temp_ids = get_product_ids(directory.path)
    product_ids += temp_ids
    
len(set(product_ids)

79